In [1]:
import numpy as np
import pandas as pd 
import sqlalchemy as sa
from sqlalchemy import create_engine

In [2]:
import matplotlib.pyplot as ply
import seaborn as sn
%matplotlib inline

In [3]:
Username= 'Admin'
Password= 'amazing_people'
Host='alphacode-explore.ccwgqdqrrmvt.eu-west-1.rds.amazonaws.com'
Port='1433'
Database='Tebogo'


In [4]:
connection = create_engine(
    'mssql+pymssql://' +
    Username + ':' + Password + '@' + Host + ':' + Port + '/' + Database
)


In [5]:
sql_query1= "Select* from claims_data"
sql_query2= "Select* from test_set"

In [6]:
df= pd.read_sql_query(sql_query1, connection)


In [7]:
df.head()

,age,sex,bmi,steps,children,smoker,region,insurance_claim,claim_amount
0,19,female,27.9,3009,0,yes,southwest,yes,16884.924
1,18,male,33.77,3008,1,no,southeast,yes,1725.5523
2,28,male,33,3009,3,no,southeast,no,0
3,33,male,22.705,10009,0,no,northwest,no,0
4,32,male,28.88,8010,0,no,northwest,yes,3866.8552


In [8]:
df2 = pd.read_sql_query(sql_query2, connection)

In [9]:
df2.head()

,row_id,age,sex,bmi,steps,children,smoker,region
0,1,27,male,28.88,8049,0,no,southwest
1,2,20,female,26.265,7970,3,yes,southwest
2,3,32,female,26.025,2973,0,no,southwest
3,4,19,female,22.085,10427,1,yes,southwest
4,5,20,female,28.685,3972,0,yes,southwest


Logistic Regression(Claims Data)

In [10]:
#Lables
y= df['insurance_claim']

#Features

X = df.drop(['insurance_claim','claim_amount','region','sex','smoker'],axis=1)
X.head()

,age,bmi,steps,children
0,19,27.9,3009,0
1,18,33.77,3008,1
2,28,33,3009,3
3,33,22.705,10009,0
4,32,28.88,8010,0


In [11]:
X.shape

(1338, 4)

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [13]:
X_train, X_test, y_train, y_test = train_test_split(
 X, y, test_size=0.2, random_state=47)

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

In [15]:
knn = KNeighborsClassifier()

In [16]:
lm = LogisticRegression(solver='lbfgs')
knn = KNeighborsClassifier()

In [17]:
lm.fit(X_train,y_train)
knn.fit(X_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')

In [18]:
lm.intercept_[0]

-0.005011514012553102

In [19]:
pred_lm = lm.predict(X_test)
# pred_knn = knn.predict(X_test)

In [20]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report


In [21]:
labels = ['No', 'Yes']

print('Confusion Matrix')
pd.DataFrame(data=confusion_matrix(y_test, pred_lm), index=labels, columns=labels)

# print('Classification Report')
# print(classification_report(y_test, pred_knn, target_names=['No', 'Yes']))

Confusion Matrix


,No,Yes
No,77,25
Yes,25,141


In [22]:
from sklearn.metrics import classification_report

In [23]:
print('Classification Report')
print(classification_report(y_test, pred_lm, target_names=['No', 'Yes']))

Classification Report
              precision    recall  f1-score   support

          No       0.75      0.75      0.75       102
         Yes       0.85      0.85      0.85       166

   micro avg       0.81      0.81      0.81       268
   macro avg       0.80      0.80      0.80       268
weighted avg       0.81      0.81      0.81       268



Train

In [24]:
knn_1 = KNeighborsClassifier(n_neighbors=1)
knn_10 = KNeighborsClassifier(n_neighbors=10)
knn_25 = KNeighborsClassifier(n_neighbors=25)

In [25]:
knn_1.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=1, p=2,
           weights='uniform')

In [26]:
knn_10.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=10, p=2,
           weights='uniform')

In [27]:
knn_25.fit(X_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=25, p=2,
           weights='uniform')

Predict

In [28]:
pred_knn_1 = knn_1.predict(X_test)
pred_knn_10 = knn_10.predict(X_test)
pred_knn_25 = knn_25.predict(X_test)

Test

In [29]:
labels = ['No', 'Yes']

pred = {'k=1':pred_knn_1, 'k=10':pred_knn_10, 'k=25':pred_knn_25}

for k,v in pred.items():
    print('{}'.format(k))
    print(pd.DataFrame(data=confusion_matrix(y_test, v), index=labels, columns=labels))
    print('\n')

k=1
     No  Yes
No   68   34
Yes  35  131


k=10
     No  Yes
No   82   20
Yes  36  130


k=25
     No  Yes
No   64   38
Yes  23  143




Classification Report

In [30]:
pred = {'k=1':pred_knn_1, 'k=10':pred_knn_10, 'k=25':pred_knn_25}

for k,v in pred.items():
    print('{}'.format(k))
    print(classification_report(y_test, v, target_names=['No', 'Yes']))
    print('\n')

k=1
              precision    recall  f1-score   support

          No       0.66      0.67      0.66       102
         Yes       0.79      0.79      0.79       166

   micro avg       0.74      0.74      0.74       268
   macro avg       0.73      0.73      0.73       268
weighted avg       0.74      0.74      0.74       268



k=10
              precision    recall  f1-score   support

          No       0.69      0.80      0.75       102
         Yes       0.87      0.78      0.82       166

   micro avg       0.79      0.79      0.79       268
   macro avg       0.78      0.79      0.78       268
weighted avg       0.80      0.79      0.79       268



k=25
              precision    recall  f1-score   support

          No       0.74      0.63      0.68       102
         Yes       0.79      0.86      0.82       166

   micro avg       0.77      0.77      0.77       268
   macro avg       0.76      0.74      0.75       268
weighted avg       0.77      0.77      0.77       268




# Decision Tree

In [31]:
from sklearn.tree import DecisionTreeClassifier

In [32]:
tree = DecisionTreeClassifier(random_state =47)

In [33]:
tree.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=47,
            splitter='best')

# Predicting

In [34]:
pred_tree = tree.predict(X_test)

# Testing

In [35]:
print('Decision Tree - Classification Report')
print(classification_report(y_test, pred_tree, target_names=['No', 'Yes']))

Decision Tree - Classification Report
              precision    recall  f1-score   support

          No       0.86      0.86      0.86       102
         Yes       0.92      0.92      0.92       166

   micro avg       0.90      0.90      0.90       268
   macro avg       0.89      0.89      0.89       268
weighted avg       0.90      0.90      0.90       268



# Random Forest

In [36]:
from sklearn.ensemble import RandomForestClassifier

In [37]:
forest = RandomForestClassifier(n_estimators=100, random_state=47)

In [38]:
forest.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=47, verbose=0, warm_start=False)

Predicting 

In [39]:
pred_forest = forest.predict(X_test)

In [40]:
labels = ['No', 'Yes']

print('Random Forest - Confusion Matrix')
pd.DataFrame(data=confusion_matrix(y_test, pred_forest), index=labels, columns=labels)

Random Forest - Confusion Matrix


,No,Yes
No,93,9
Yes,18,148


In [41]:
print('Random Forest - Classification Report')
print(classification_report(y_test, pred_forest, target_names=['No', 'Yes']))

Random Forest - Classification Report
              precision    recall  f1-score   support

          No       0.84      0.91      0.87       102
         Yes       0.94      0.89      0.92       166

   micro avg       0.90      0.90      0.90       268
   macro avg       0.89      0.90      0.89       268
weighted avg       0.90      0.90      0.90       268



Train

In [42]:
# 5 trees in forest
forest_1 = RandomForestClassifier(n_estimators=5, random_state=47)

# 25 trees in forest
forest_2 = RandomForestClassifier(n_estimators=25, random_state=47)

# 100 trees in forest
forest_3 = RandomForestClassifier(n_estimators=100, random_state=47)

In [43]:
forest_1.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=5, n_jobs=None,
            oob_score=False, random_state=47, verbose=0, warm_start=False)

In [44]:
forest_2.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=25, n_jobs=None,
            oob_score=False, random_state=47, verbose=0, warm_start=False)

In [45]:
forest_3.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=47, verbose=0, warm_start=False)

Predict

In [46]:
pred_forest_1 = forest_1.predict(X_test)
pred_forest_2 = forest_2.predict(X_test)
pred_forest_3 = forest_3.predict(X_test)

Testing

In [47]:
labels = ['No', 'Yes']

pred = {'trees = 5':pred_forest_1,
        'trees = 25':pred_forest_2,
        'trees = 100':pred_forest_3}

for k,v in pred.items():
    print('{}'.format(k))
    print(pd.DataFrame(data=confusion_matrix(y_test, v), index=labels, columns=labels))
    print('\n')

trees = 5
     No  Yes
No   89   13
Yes  18  148


trees = 25
     No  Yes
No   91   11
Yes  15  151


trees = 100
     No  Yes
No   93    9
Yes  18  148




Classification Report

In [48]:
pred = {'trees = 5':pred_forest_1,
        'trees = 25':pred_forest_2,
        'trees = 100':pred_forest_3}

for k,v in pred.items():
    print('{}'.format(k))
    print(classification_report(y_test, v, target_names=['No', 'Yes']))
    print('\n')

trees = 5
              precision    recall  f1-score   support

          No       0.83      0.87      0.85       102
         Yes       0.92      0.89      0.91       166

   micro avg       0.88      0.88      0.88       268
   macro avg       0.88      0.88      0.88       268
weighted avg       0.89      0.88      0.88       268



trees = 25
              precision    recall  f1-score   support

          No       0.86      0.89      0.88       102
         Yes       0.93      0.91      0.92       166

   micro avg       0.90      0.90      0.90       268
   macro avg       0.90      0.90      0.90       268
weighted avg       0.90      0.90      0.90       268



trees = 100
              precision    recall  f1-score   support

          No       0.84      0.91      0.87       102
         Yes       0.94      0.89      0.92       166

   micro avg       0.90      0.90      0.90       268
   macro avg       0.89      0.90      0.89       268
weighted avg       0.90      0.90    

# SVM Classification

In [49]:
from sklearn.svm import SVC

In [50]:
svm = SVC(random_state=47)

In [51]:
svm.fit(X_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=47,
  shrinking=True, tol=0.001, verbose=False)

Predicting

In [52]:
pred_svm = svm.predict(X_test)

Testing

In [53]:
labels = ['No', 'Yes']

print('SVM - Confusion Matrix')
pd.DataFrame(data=confusion_matrix(y_test, pred_svm), index=labels, columns=labels)

SVM - Confusion Matrix


,No,Yes
No,64,38
Yes,16,150


In [54]:
print('SVM - Classification Report')
print(classification_report(y_test, pred_svm, target_names=['No', 'Yes']))

SVM - Classification Report
              precision    recall  f1-score   support

          No       0.80      0.63      0.70       102
         Yes       0.80      0.90      0.85       166

   micro avg       0.80      0.80      0.80       268
   macro avg       0.80      0.77      0.78       268
weighted avg       0.80      0.80      0.79       268



Train

In [55]:
# kernel=linear, C=1, gamma=0.1
svm_1 = SVC(kernel='linear', C=1, gamma=0.1, random_state=47)

# kernel=rbf, C=10, gamma=0.01
svm_2 = SVC(kernel='rbf', C=10, gamma=0.01, random_state=47)

# kernel=poly, C=100, gamma=1
svm_3 = SVC(kernel='poly', C=100, gamma=1, random_state=47)

In [56]:
# svm_1.fit(X_train, y_train)

In [57]:
# svm_2.fit(X_train, y_train)

In [58]:
# svm_3.fit(X_train, y_train)

Predict

In [59]:
# pred_svm_1 = svm_1.predict(X_test)
# pred_svm_2 = svm_2.predict(X_test)
# pred_svm_3 = svm_3.predict(X_test)

Test

In [60]:
# labels = ['No', 'Yes']

# pred = {'kernel=linear, C=1, gamma=0.1':pred_svm_1,
#         'kernel=rbf, C=10, gamma=0.01':pred_svm_2,
#         'kernel=poly, C=100, gamma=1':pred_svm_3}

# for k,v in pred.items():
#     print('{}'.format(k))
#     print(pd.DataFrame(data=confusion_matrix(y_test, v), index=labels, columns=labels))
#     print('\n')

In [61]:
# pred = {'kernel=linear, C=1, gamma=0.1':pred_svm_1,
#         'kernel=rbf, C=10, gamma=0.01':pred_svm_2,
#         'kernel=poly, C=100, gamma=1':pred_svm_3}

# for k,v in pred.items():
#     print('{}'.format(k))
#     print(classification_report(y_test, v, target_names=['No', 'Yes']))
#     print('\n')

# Conclusion 

The Model that has the highest F1 score is the Decision tree, with an F10

In [62]:
sql_query ='Select* from test_set'

In [63]:
df1=pd.read_sql_query(sql_query,connection)

In [64]:
df1.head(25)

,row_id,age,sex,bmi,steps,children,smoker,region
0,1,27,male,28.88,8049,0,no,southwest
1,2,20,female,26.265,7970,3,yes,southwest
2,3,32,female,26.025,2973,0,no,southwest
3,4,19,female,22.085,10427,1,yes,southwest
4,5,20,female,28.685,3972,0,yes,southwest
5,6,24,female,23.8,8789,0,yes,southwest
6,7,58,male,35.245,3095,1,no,southeast
7,8,46,male,24.7,8729,1,no,northwest
8,9,42,female,26.08,8439,0,no,southwest
9,10,21,female,52.58,3636,0,no,southwest


In [65]:
rwid = df1.row_id.values

In [66]:
X = df1.drop(['region','row_id','sex','smoker'],axis=1)
X.head()

,age,bmi,steps,children
0,27,28.88,8049,0
1,20,26.265,7970,3
2,32,26.025,2973,0
3,19,22.085,10427,1
4,20,28.685,3972,0


In [67]:
 prd = tree.predict(X)

In [68]:
submission = pd.DataFrame({'row_id':rwid,'predictions':prd})

In [3]:
submission.head()

NameError: name 'submission' is not defined

In [ ]:
FinalModel.to_sql('classification_model_predict', con=connection, if_exists='append', index=True,method='multi',chunksize=500)


In [ ]:
#Review tables in Database
connection.table_names()

In [71]:
submission.to_csv('Tebogo classification_model_prediction')